In [ ]:
# The following is from "big run"
import sys
sys.path.append('..')
from lib import *
import pandas as pd
import numpy as np

# Setup
lsa_np = np.load('../data/parsed/lsa_output.npy')

metadata = pd.read_pickle('../data/parsed/pickles/pickled_data_test.pickle')
metadata = metadata.loc[metadata['Scenario'] == '401']
metadata = metadata.reset_index(drop=True)

lsa_df = pd.DataFrame(lsa_np)

df = pd.concat([metadata, lsa_df], axis=1, join_axes=[metadata.index])
df = df.loc[df['Label'] != '-1']
df = df.reset_index(drop=True)

cat_features = ['To','From']
features = list(range(100))
features.extend(cat_features + ['Date'])

df = df[features + ['Label'] + ['ID']]


# Built-in incremental learning vs trees training on larger initial sizes

## Initializing control variables

In [ ]:
# now using params from sweep

n_trees = 32
tree_depth = 70
random_seed = 42
n_max_features = 90
cat_features = ['To', 'From']

## Forests Trained on increasing datasets

In [ ]:
forest_100 = RNF(df[:100], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)
forest_200 = RNF(df[:200], n_trees, tree_depth, random_seed, n_max_features, 200, cat_features)
forest_300 = RNF(df[:300], n_trees, tree_depth, random_seed, n_max_features, 300, cat_features)
forest_400 = RNF(df[:400], n_trees, tree_depth, random_seed, n_max_features, 400, cat_features)
forest_500 = RNF(df[:500], n_trees, tree_depth, random_seed, n_max_features, 500, cat_features)
incremental_forests = [forest_100, forest_200, forest_300, forest_400, forest_500]

In [ ]:
for forest in incremental_forests:
    forest.fit_parallel()

In [ ]:
# evaluation
for forest in incremental_forests:
    print(evalStats(forest.predict_parallel(df[-100:])[1], df[-100:]), end='\n\n')

100, 200, ..., 500 document - trained forests
Param

##  Incremental Forests

In [ ]:
incremental_forest = RNF(df[0:100], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)

In [ ]:
incremental_forest.fit_parallel()
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

In [ ]:
incremental_forest.update(df[100:200])
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

incremental_forest.update(df[200:300])
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

incremental_forest.update(df[300:400])
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

incremental_forest.update(df[400:500])
print(evalStats(incremental_forest.predict_parallel(df[-100:])[1], df[-100:]))

In [ ]:
incremental_forest.train_data.shape[0]

In [ ]:
forest_500.train_data.shape[0]

# Testing limited core usage

In [ ]:
f = RNF(df[0:500], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)
f.fit_parallel()

In [ ]:
f.trees[20].visualize()

# Testing 400 incremented vs 400 initial trained

In [ ]:
# incremental forest initially trained on 100 rows
inc_forest = RNF(df[0:100], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)
inc_forest.fit_parallel();

In [ ]:
init_forest = RNF(df[:-100], n_trees, tree_depth, random_seed, n_max_features, 400, cat_features)
init_forest.fit_parallel();

# Comparing Query Strategies

## Query by Committee
Asking for labeling on rows with the most amount of disagreement (between trees)

In [ ]:
committee_rnf = RNF(df[:500], n_trees, tree_depth, random_seed, n_max_features, 500, cat_features)
committee_rnf.fit_parallel()

In [ ]:
predictions = committee_rnf.predict_parallel(df[-100:])

In [ ]:
# agreements = predictions[0]
def entropy(pred):
    s = 0
    for x in pred[0]:
        if x != 0:
            s += x * math.log(x)
    return (-1 * s, pred[1])

# entropies = sorted(map(entropy, zip(agreements, predictions[2])), reverse=True)

In [ ]:
def committee_increment():
    # initial training
    committee_rnf = RNF(df[:100], n_trees, tree_depth, random_seed, n_max_features, 100, cat_features)
    committee_rnf.fit_parallel()
    
    labeled_ids = df.loc[:100, 'ID'].values
    test_ids = df.loc[-100:, 'ID'].values
    labeled_ids = np.append(labeled_ids, test_ids)
    test_results = committee_rnf.predict_parallel(df.loc[df['ID'].isin(test_ids)])
    print(evalStats(test_results[1], df.loc[df['ID'].isin(test_ids)]))
    for i in range(5):
        # initial train scores

        unlabeled_predict = committee_rnf.predict_parallel(df.loc[np.logical_not(df['ID'].isin(test_ids))])

        to_label = sorted(map(entropy, zip(unlabeled_predict[0], unlabeled_predict[2])), reverse=True)[:100]

        to_label_ids = [x[1] for x in to_label]

        labeled_ids = np.append(labeled_ids, to_label_ids)

        increment_df = df.loc[df["ID"].isin(to_label_ids)]
        
        committee_rnf.update(increment_df)
    
        test_results = committee_rnf.predict_parallel(df.loc[df['ID'].isin(test_ids)])
        print(evalStats(test_results[1], df.loc[df['ID'].isin(test_ids)]))
        
committee_increment()

## Moving towards training data homogeneity
Trying to get Forst DF's relevant/irrelevant distribution to 1:1